In [21]:
%load_ext autoreload
%autoreload 2
from channel_utils import Channel
from encoder_utils import Encoder
from decoder_utils import Decoder
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
L = 10 # Number of previous bits used
N = 18 # Number of output bits per input bit

In [23]:
# filenames
original_file  = '80char.txt'
encoded_file   = 'encoded.txt'
channel_output = 'channel_output.txt'
decoded_file   = 'decoded.txt'

In [24]:
def generate_generator_matrix():
    """
    Creates the generator matrix.
    We make sure that for each generator_polynomial (A line of the generator matrix), the first and the last bits are set to 1
    This is because we at least want b_i and b_{i-L} to be used to use the whole range L. The remaining coefficients are chosen
    at random as no method exists to optimally pick them deterministically.
    """
    generator_matrix = np.random.choice([0, 1], size=(N, L+1))
    generator_matrix[:,0] = 1
    generator_matrix[:,L] = 1
    return generator_matrix
generator_matrix = generate_generator_matrix()
print('Generator matrix :\n', generator_matrix)

Generator matrix :
 [[1 1 1 1 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 0 1 1]
 [1 1 0 0 1 1 0 0 0 0 1]
 [1 0 1 1 1 0 0 1 0 0 1]
 [1 0 0 0 0 1 1 1 1 0 1]
 [1 0 0 1 0 0 0 1 0 1 1]
 [1 0 0 1 0 1 0 0 0 1 1]
 [1 1 1 1 0 0 0 1 1 1 1]
 [1 0 1 0 0 0 1 0 1 0 1]
 [1 0 1 1 1 1 1 0 1 1 1]
 [1 0 1 1 0 0 0 1 0 0 1]
 [1 1 1 0 1 1 1 0 0 1 1]
 [1 1 1 1 0 0 1 1 1 1 1]
 [1 1 0 0 0 1 1 1 0 0 1]
 [1 0 1 0 1 0 1 0 0 0 1]
 [1 1 1 1 1 0 0 0 1 0 1]
 [1 1 0 1 0 1 1 1 1 1 1]
 [1 0 1 1 1 0 1 1 0 1 1]]


In [25]:
encoder = Encoder(input_file=original_file,  output_file=encoded_file, generator_matrix=generator_matrix, verbose=False)
decoder = Decoder(input_file=channel_output, output_file=decoded_file, generator_matrix=generator_matrix, verbose=False)

In [26]:
# Encode

encoder.encode()

Original file : b'Hello world, planet, solar system all the way to A-Centuri and the entire galaxy'

Hello world, planet, solar system all the way to A-Centuri and the entire galaxy
Total number of bits used : 23040


In [27]:
# AWGN + erasure channel

!python client\client.py --input_file {encoded_file} --output_file {channel_output} --srv_hostname iscsrv72.epfl.ch --srv_port 80

In [28]:
# Decode

decoder.decode()

Compact_values_length : 11520
nb_trellis_sections : 640
Trellis height : 1024
Trellis width  : 641
Decoding ...
bytearray(b'Hello world, planet, solar system all the way to A-Centuri and the entire galaxy')


In [6]:
def run_all():
    generator_matrix = generate_generator_matrix()
    channel = Channel(changingIndex=True, verbose=False)
    encoder = Encoder(input_file='80char.txt', output_file='encoded.txt', generator_matrix=generator_matrix, verbose=False)
    decoder = Decoder(input_file='noisy.txt', output_file='decoded.txt', generator_matrix=generator_matrix, verbose=False)
    encoder.encode()
    channel.sendFileWithOutput(inputFileName='encoded.txt', outputFileName='noisy.txt')
    decoder.decode()

In [9]:
for i in range(10):
    print(f'Run {i+1} :')
    run_all()
    print()
    print()

Run 1 :
Original file : b'Hello world, planet, solar system all the way to A-Centuri and the entire galaxy'

Hello world, planet, solar system all the way to A-Centuri and the entire galaxy
Total number of bits used : 23040
Channel sending with H value 0 :
[ 0. -1. -1. ...  0. -1. -1.]
Compact_values_length : 11520
nb_trellis_sections : 640
Trellis height : 1024
Trellis width  : 641
Decoding ...
bytearray(b'Hello world, planet, solar system all the way to A-Centuri and the entire galaxy')


Run 2 :
Original file : b'Hello world, planet, solar system all the way to A-Centuri and the entire galaxy'

Hello world, planet, solar system all the way to A-Centuri and the entire galaxy
Total number of bits used : 23040
Channel sending with H value 0 :
[ 0. -1. -1. ...  0. -1. -1.]
Compact_values_length : 11520
nb_trellis_sections : 640
Trellis height : 1024
Trellis width  : 641
Decoding ...
bytearray(b'Hello world, planet, solar system all the way to A-Centuri and the entire galaxy')


Run 3 :
